In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils import options
from dataset import predset
from model import modelset
import pandas as pd
import torch
from utils import utils
from model import lstm
from importlib import reload
reload(utils)
reload(lstm)
reload(predset)
reload(modelset)

Make prediction on your files in folder test_data_scientist :

In [ ]:
class Opt():
    def __init__(self):
        self.dataroot="./test_data_scientist/"
        self.device=0
opt = Opt()
if opt.device==1:
    device=torch.device("cuda:0")
    print("Using GPU... \n")
else:
    device=torch.device("cpu")
    print("Using CPU... \n")
pred_set=predset.PredSet()
pred_set.initialize(opt)
model_set=modelset.ModelSet()
model_set.initialize(device)
model_set.predict(pred_set)

# To use with Training & Validation dataset

In [ ]:
sample=pred_set.dataset_to_nn.samples[6]
sample

In [ ]:
plt.plot(sample["x"])
plt.plot(utils.inversenorm(np.concatenate((sample["annual_x"],sample["annual_y"])),sample["max_turn"],sample["min_turn"]))
plt.show()

In [ ]:
truth=pred_set.pred_df[(pred_set.pred_df["but_num_business_unit"]==sample["bu_num"])
                        & (pred_set.pred_df["dpt_num_department"]==sample["dep"])
                        ].sort_values(by=["day_id"]).reset_index(drop=True)["turnover"].to_numpy()
plt.plot(np.concatenate((sample["x"],sample["pred"])),color="red")
plt.plot(np.concatenate((sample["x"],truth)),color="cyan")
plt.plot(sample["x"],color="blue")
plt.plot(utils.inversenorm(np.concatenate((sample["annual_x"],sample["annual_y"])),sample["max_turn"],sample["min_turn"]),color="green")
plt.show()

In [ ]:
device= torch.device("cpu")
dep=sample["dep"]
model = lstm.LSTM_Turnover(True,hidden_size=64,num_of_layer=4).to(device)
model.load_state_dict(torch.load("./model/models_weights/weights_{}.pth".format(dep),map_location='cpu')['model_state_dict'])
man_pred=model( torch.Tensor(sample["normed_x"]).float().unsqueeze(0).to(device),
                    torch.Tensor(sample["annual_x"]).float().unsqueeze(0).to(device),
                    torch.Tensor(sample["annual_y"]).float().unsqueeze(0).to(device),
                    8)
man_pred=man_pred[0,-8:].detach().numpy()
man_pred_un=utils.inversenorm(man_pred,sample["max_turn"],sample["min_turn"])

In [ ]:
device= torch.device("cpu")
dep=sample["dep"]
model = lstm.LSTM_Turnover(True,hidden_size=64,num_of_layer=4).to(device)
model.load_state_dict(torch.load("./model/models_weights/weights_{}.pth".format(dep),map_location='cpu')['model_state_dict'])
man_pred=model( torch.Tensor(sample["normed_x"]).float().unsqueeze(0).to(device),
                    torch.Tensor(sample["annual_x"]).float().unsqueeze(0).to(device),
                    torch.Tensor(sample["annual_y"]).float().unsqueeze(0).to(device),
                    8)
man_pred=man_pred[0,-8:].detach().numpy()
man_pred_un=utils.inversenorm(man_pred,sample["max_turn"],sample["min_turn"])

In [ ]:
device= torch.device("cpu")
dep=sample["dep"]
model = lstm.LSTM_Turnover(True,hidden_size=64,num_of_layer=4).to(device)
model.load_state_dict(torch.load("./model/models_weights/weights_{}.pth".format(dep),map_location='cpu')['model_state_dict'])
man_pred=model( torch.Tensor(sample["normed_x"]).float().unsqueeze(0).to(device),
                    torch.Tensor(sample["annual_x"]).float().unsqueeze(0).to(device),
                    torch.Tensor(sample["annual_y"]).float().unsqueeze(0).to(device),
                    8)
man_pred=man_pred[0,-8:].detach().numpy()
man_pred_un=utils.inversenorm(man_pred,sample["max_turn"],sample["min_turn"])

In [ ]:
pred_set.write_prediction()

In [ ]:
res=pred_set.results_df.sort_values(by=["day_id"])
y=res[(res["but_num_business_unit"]==sample["bu_num"]) & (res["dpt_num_department"]==sample["dep"])]["results"].to_numpy()
y_t=res[(res["but_num_business_unit"]==sample["bu_num"]) & (res["dpt_num_department"]==sample["dep"])]["turnover"].to_numpy()
plt.plot(y)
plt.plot(y_t)

# To use with Training & Test datasets

In [ ]:
sample=None
for elem in pred_set.dataset_to_nn.samples:
    if (elem["bu_num"]==4) & (elem["dep"]==117):
        sample=elem
sample


In [ ]:
plt.plot(utils.inversenorm(np.concatenate((sample["annual_x"],sample["annual_y"])),sample["max_turn"],sample["min_turn"]))
plt.plot(np.concatenate((sample["x"],sample["pred"])),color="red",linewidth=4)
plt.plot(sample["x"])
plt.show()